# Aula 14 &mdash; Regressão linear (Parte 2)

Renato Vimieiro

rv2 {em} cin.ufpe.br

maio 2017

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from scipy.stats import linregress, t

% matplotlib inline

In [2]:
ads = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv",
                  usecols=[1,2,3,4])

## Introdução

Iniciamos na aula passada a discussão sobre regressão linear. Vimos que o objetivo central é encontrar um modelo capaz de predizer valores para a variável dependente dados os valores de uma variável independente. Vimos também que, além de predizer valores, o modelo pode ser usada para avaliar (se existe) a relação entre as variáveis dependente e independente. 

Levantamos na aula passada as seguintes perguntas:

1. Em cada caso, qual a reta que melhor descreve os dados? &checkmark;
2. Quão forte é essa relação entre publicidade e vendas? &checkmark;
4. Qual a precisão do modelo e da influência dos meios nas vendas? &#8723;
3. Das três mídias investidas, qual é a que mais contribui para o aumento de vendas?
5. Existe alguma interação entre as diferentes mídias? O efeito de cada uma delas é isolado ou elas se inter-influenciam?

O foco da aula anterior foi nos modelos com apenas uma variável independente. Isso fez com que analisássemos o investimento em publicidade em cada meio de forma isolada no exemplo usado. Nessa aula focaremos nos modelos com múltiplas variáveis dependentes, na inclusão de variáveis categóricas no modelo e o uso de funções não-lineares como variáveis dependentes.

## Regressão Linear Múltipla

Nosso interesse nessa aula é em ajustar um hiperplano ao conjunto de dados de modo a predizer valores para a variável dependente com o menor erro possível. Nosso modelo é generalizado, dessa forma, para

$$\hat{y} = \hat{\beta_0} + \hat{\beta_1}X_1 + \hat{\beta_2}X_2 + \ldots + \hat{\beta_p}X_p$$

Se incluirmos uma variável $X_0=1$ a todos os exemplos do conjunto de dados, podemos representar o modelo em notação matricial como

$$\hat{y} = X\hat{\beta}$$

Mais uma vez os melhores coeficientes são aqueles que minimizam a soma dos quadrados dos erros

$$RSS = (y - X\hat{\beta})^T(y - X\hat{\beta})$$.

O erro é minimizado igualando-se as derivadas parciais de RSS com respeito a $\hat{\beta}$. O resultado nos dá a seguinte equação:

\begin{equation*}
\hat{\beta} = (X^TX)^{-1}X^Ty
\end{equation*}

Portanto, o sistema só tem solução se $X^TX$ for inversível. Isso pode ser um problema para dados de alta dimensionalidade, sobretudo os de biologia, em que $n < p$, e em casos de correlação entre as variáveis independentes. Na maior parte dos casos, o número de atributos é (bem) menor que o número de objetos, logo esse é um problema menos grave. No entanto, o problema da multicolinearidade é recorrente. Nesses casos, o método dos mínimos quadrados não é aplicável ou não confiável.

Voltando ao exemplo, podemos ajustar o modelo tomando o investimento em publicidade nas três mídias como variáveis independentes e o total de vendas como variável dependente.

In [31]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
model = smf.ols("Sales ~ TV + Radio + Newspaper",data=ads)
result = model.fit()

In [32]:
pd.concat([result.params,result.bse,result.tvalues,result.pvalues],
          axis=1, keys=['coef','SE','t','p-value'])

,coef,SE,t,p-value
Intercept,2.938889,0.311908,9.422288,1.267295e-17
TV,0.045765,0.001395,32.808624,1.509960e-81
Radio,0.188530,0.008611,21.893496,1.505339e-54
Newspaper,-0.001037,0.005871,-0.176715,8.599151e-01


Interpretamos o nosso modelo, agora, como a seguir. Na ausência de qualquer investimento em publicidade, teremos em média uma de 3 unidades. Mantendo-se o investimento em rádio e jornal fixos, o investimento de \$1000 em TV resultará no aumento de vendas em 46 unidades. Já se mantivermos fixos os valores de TV e jornal, teremos um aumento de 189 unidades se investirmos \$1000 em publicidade no rádio.

Como vimos pelo gráfico na aula passada, o investimento em publicidade em jornal impresso não é significativo para aumentar o volume de vendas. Isso fica bem claro pelo p-value do coeficiente no modelo múltiplo. Contudo, caso tivéssemos tomado os modelos individualmente, teríamos uma outra percepção. Vejamos

In [65]:
model2 = smf.ols("Sales ~ Newspaper",data=ads)
result2 = model2.fit()
pd.concat([result2.params,result2.bse,result2.tvalues,result2.pvalues],
          axis=1, keys=['coef','SE','t','p-value'])

,coef,SE,t,p-value
Intercept,12.351407,0.621420,19.876096,4.713507e-49
Newspaper,0.054693,0.016576,3.299591,1.148196e-03


O p-value do coeficiente quando tomado isoladamente é pequeno e, portanto, rejeitamos a hipótese nula de que não existe relação entre o aumento de vendas e a publicidade em jornal.

A razão para essa divergência encontra-se no fato de existir uma correlação entre o investimento em rádio e jornal. Veja a tabela:

In [67]:
ads.corr()

,TV,Radio,Newspaper,Sales
TV,1.000000,0.054809,0.056648,0.782224
Radio,0.054809,1.000000,0.354104,0.576223
Newspaper,0.056648,0.354104,1.000000,0.228299
Sales,0.782224,0.576223,0.228299,1.000000


Isso indica que o aumento no investimento em publicidade em rádio é acompanhado no aumento no investimento em jornal. Quando analisamos a relação entre publicidade em jornal e aumento de vendas isoladamente, os resultados são mascarados por essa informação, dando a falsa impressão de que foi o investimento em jornal que resultou no aumento de vendas. Porém, quando tomamos todas as variáveis ao mesmo tempo, os investimentos em rádio e TV também são considerados, mostrando que a influência do investimento em jornal não é significativa frente às outras mídias.

## Existe relação entre variáveis dependente e independentes?

O p-value de cada coeficiente está associado ao teste da hipótese nula de que o coeficiente é zero. No caso da regressão múltipla, temos de avaliar a hipótese nula de todos os coeficientes serem zero ao mesmo tempo para confirmar ou não a ausência de relação entre a variável dependente e as independentes. Nesse caso usamos o *teste F* que é baseado na probabilidade de observação de uma estatística F tão extrema quanto à observada. A estatística F avalia a razão entre a variância explicada e a não explicada, e é dada pela fórmula:

\begin{equation*}
F = \frac{(TSS-RSS)/p}{RSS/(n-p-1)}
\end{equation*}
$TSS = \sum (y_i-\bar{y})^2$ é a variância total, e $RSS$ é a soma dos quadrados dos resíduos.

Felizmente a biblioteca já computa esses valores e não precisamos computar nem a estatística F nem o p-value. Esses valores para o nosso exemplo são dados a seguir. De fato, a biblioteca mostra um conjunto bem completo de estatísticas do modelo.

In [68]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Wed, 10 May 2017   Prob (F-statistic):           1.58e-96
Time:                        10:41:12   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
Radio          0.1885      0.009     21.893      0.000       0.172       0.206
Newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Inclusão de variáveis categóricas e funções não-lineares das variáveis independentes

O modelo de regressão múltipla permite a inclusão tanto de variáveis categóricas quanto de funções da variáveis independentes. De fato, a linearidade do modelo está em relação aos parâmetros e não em cada variável independente. Em outras palavras, podemos criar novas variáveis, adicioná-las aos dados, e ajustar o modelo considerando essas novas variáveis. 

Para ilustrar esses conceitos, vamos trabalhar com um outro conjunto de dados sobre inadimplência com base em critérios sócio-econômicos. Esse exemplo também é tirado do livro ISL.

In [82]:
credit = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Credit.csv',index_col=0)
credit.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


#### Incorporando variáveis categóricas com dois níveis

Como vemos, o conjunto possui tanto valores categóricos quanto numéricos.

Variáveis categóricas podem ser incluídas no modelo criando-se variáveis fictícias (dummy) para as categorias. Essas variáveis, chamadas de indicadores, vão conter 1 se o exemplo tiver a categoria, e zero caso contrário. Por exemplo, no caso de gênero acima, criamos uma variável fictícia $I(gênero=F)$ que contém 1 se a pessoa for uma mulher, e zero se for um homem. Nesse caso, nosso modelo se resumirá a

\begin{align}
\hat{y} = \hat{\beta_0} + \hat{\beta_1} &\quad \text{se mulher}\\
\hat{y} = \hat{\beta_0} & \quad\text{se homem}\\
\end{align}

In [84]:
model = smf.ols("Balance ~ Gender",data=credit).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1836
Date:                Wed, 10 May 2017   Prob (F-statistic):              0.669
Time:                        11:31:04   Log-Likelihood:                -3019.3
No. Observations:                 400   AIC:                             6043.
Df Residuals:                     398   BIC:                             6051.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          509.8031     33.128     15.389      0.000     444.675     574.931
Gender[T.Female]    19.7331     46.051      0.429      0.669     -70.801     110.267
==============================================================================
Omnibus:                       28.438   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.346
Skew:                           0.583   Prob(JB):                     1.15e-06
Kurtosis:                       2.471   Cond. No.                         2.66
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

O coeficiente $\hat{\beta_1}$ deve ser interpretado como a diferença média de saldo devedor entre homens e mulheres. Assim, vemos que, em média, as mulheres possuem cerca de \$20 a mais de saldo devedor que os homens nesse conjunto de dados. Como a interseção com o eixo y foi de aproximadamente \$510, temos que esse é o saldo dos homens, enquanto das mulheres é \$530.

Note que uma outra representação para as variáveis categóricas (por exemplo, -1 e 1) levaria a uma outra interpretação do modelo. Nesse caso, o coeficiente $\hat{\beta_1}$ mostraria a divergência de homens e mulheres com respeito à média global. É importante ressaltar que ambas as representações estão corretas. O que muda é somente a interpretação do modelo. No entanto, o mais usual é a primeira representação.

#### Incorporando variáveis categóricas com mais de dois níveis

Caso as variáveis categóricas tenham $k > 2$ níveis, criamos $k-1$ variáveis fictícias e procedemos da mesma forma que no exemplo anterior. Por exemplo, a variável etnia possui 3 níveis (branco, asiático e negro). Assim, criamos variáveis indicadoras para cada uma das duas categorias restantes. A outra categoria não incluída é tida como valor base (**baseline**) e os coeficientes mostraram a divergência dos valores médias para as categorias inclusas com respeito a ela.

In [85]:
model = smf.ols("Balance ~ Ethnicity",data=credit).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.04344
Date:                Wed, 10 May 2017   Prob (F-statistic):              0.957
Time:                        12:01:11   Log-Likelihood:                -3019.3
No. Observations:                 400   AIC:                             6045.
Df Residuals:                     397   BIC:                             6057.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                531.0000     46.319     11.464      0.000     439.939     622.061
Ethnicity[T.Asian]       -18.6863     65.021     -0.287      0.774    -146.515     109.142
Ethnicity[T.Caucasian]   -12.5025     56.681     -0.221      0.826    -123.935      98.930
==============================================================================
Omnibus:                       28.829   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.395
Skew:                           0.581   Prob(JB):                     1.13e-06
Kurtosis:                       2.460   Cond. No.                         4.39
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Dessa forma, interpretamos os coeficientes como a seguir. Os negros possuem saldo devedor em média de \$531, enquanto os asiáticos devem em média \$18.69 a menos que eles, e os brancos \$12.50. 

Observamos que os p-values associados tanto aos coeficientes individualmente quanto à eles em conjunto são bastante altos. Isso nos mostra que o saldo devedor não está diretamente relacionado à etnia da pessoa. O mesmo pode ser dito em relação ao gênero.

#### Adicionando funções das variáveis independentes

Podemos utilizar a técnica para ajustar modelos não lineares fazendo transformações nos dados. Por exemplo, podemos ajustar um modelo polinomial de grau $k$, criando-se novas variáveis $X_i = X^i$ para $i \leq k$. O mesmo vale exponenciais ($y = \beta_0e^{\beta_1X}$ é transformado em $\log{y} = \log{\beta_0} + \beta_1X$), ou logarítmicas ($y = \beta_0 + \beta_1\log{X}$ é transformada para $e^y = e^{\beta_0} + e^{\beta_1}X$), e outras linearizações que se mostrarem adequadas.

Uma das utilidades dessa generalidade é a inclusão de funções que expressem a interação entre as variáveis independentes. Por exemplo, no caso do exemplo de publicidade, pode ser que exista interação entre a publicidade em dois veículos. Logo, investir uma certa quantidade isoladamente em TV ou rádio, não retornará o mesmo que distribuir o investimento nesses dois meios. 

Podemos adicionar, por exemplo, o produto de rádio e TV como forma de considerar a possível interação entre esses dois meios. O modelo resultante seria

$$Sales = \hat{\beta_0} + \hat{\beta_1}TV + \hat{\beta_2}Radio + \hat{\beta_3}TV\times Radio$$

Logo, o modelo pode ser reescrito como

$$Sales = \hat{\beta_0} + (\hat{\beta_1} + \hat{\beta_3}Radio)TV + \hat{\beta_2}Radio  = \hat{\beta_0} + \hat{\beta_1}^\prime TV + \hat{\beta_2}Radio$$
onde $\hat{\beta_1}^\prime = \hat{\beta_1} + \hat{\beta_3}Radio$.

Nesse caso, como $\hat{\beta_1}^\prime$ também leva em consideração mudanças no investimento em rádio, a forma como o investimento em TV afeta o total de vendas passa a incorporar o investimento em rádio.

In [86]:
model = smf.ols("Sales ~ TV + Radio + TV*Radio",data=ads).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     1963.
Date:                Wed, 10 May 2017   Prob (F-statistic):          6.68e-146
Time:                        12:50:02   Log-Likelihood:                -270.14
No. Observations:                 200   AIC:                             548.3
Df Residuals:                     196   BIC:                             561.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.7502      0.248     27.233      0.000       6.261       7.239
TV             0.0191      0.002     12.699      0.000       0.016       0.022
Radio          0.0289      0.009      3.241      0.001       0.011       0.046
TV:Radio       0.0011   5.24e-05     20.727      0.000       0.001       0.001
==============================================================================
Omnibus:                      128.132   Durbin-Watson:                   2.224
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1183.719
Skew:                          -2.323   Prob(JB):                    9.09e-258
Kurtosis:                      13.975   Cond. No.                     1.80e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.8e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

A interpretação do modelo é a seguinte. O investimento em \$1000 em TV está associado a um aumento proporcional a 19 unidades mais 1.1 unidade por real investido em rádio. Analogamente, o investimento de \$1000 na publicidade em rádio está associado a um aumento de vendas de 29 unidades mais 1.1 unidade para cada real investido em TV.